In [1]:
# import everything and define a test runner function
from importlib import reload
from helper import run_test

import block
import ecc
import helper
import network
import script
import tx

In [2]:
import time

from random import randint
from io import BytesIO

from block import Block
from bloomfilter import BloomFilter
from helper import int_to_little_endian, encode_varint, read_varint, decode_base58
from merkleblock import MerkleBlock
from network import NetworkEnvelope, SocketController
from tx import Tx

addr = 'muvpVznkBtk8rRSxLRVQRdUhsMjS7aKRne'
h160 = decode_base58(addr)
bf = BloomFilter(100, 20, 20)
bf.add(h160)

sc = SocketController('tbtc.programmingblockchain.com', testnet=True)

# construct version message
version = int_to_little_endian(70015, 4)
services = int_to_little_endian(0, 8)
timestamp = int_to_little_endian(int(time.time()), 8)
ip = b'\x00' * 18 + b'\xff\xff' + b'\x00' * 6
nonce = int_to_little_endian(randint(0, 2**64), 8)
user_agent = b'nobody'
ua = bytes([len(user_agent)]) + user_agent
latest_block = int_to_little_endian(0, 4)
# set relay to false so we don't get inv messages just yet
relay = b'\x00' 
payload = version + services + timestamp + ip + ip + nonce + ua + latest_block + relay
sc.send(b'version', payload)

# receive messages until we get a verack
sc.wait_for_commands({b'verack\x00\x00\x00\x00\x00\x00'})

# send the filterload command
sc.send(b'filterload', bf.filterload())

# ask for blocks since block X
hash_count = encode_varint(1)
last_block = bytes.fromhex('000000001439bf71cf731722c650375d0f188e528f5982c24a68a4745a809fd9')[::-1]
end_block = b'\x00' * 32
payload = version + hash_count + last_block + end_block
sc.send(b'getheaders', payload)

# wait for headers
headers_envelope = sc.wait_for_commands({b'headers\x00\x00\x00\x00\x00'})

# loop through each header and ask for the merkleblock
stream = BytesIO(headers_envelope.payload)
num_headers = read_varint(stream)
payload = encode_varint(num_headers)
for _ in range(num_headers):
    b = Block.parse(stream)
    num_txs = read_varint(stream)
    if num_txs != 0:
        raise RuntimeError('got more than 0 txs')
    # ask for merkleblock
    payload += int_to_little_endian(3, 4)
    payload += b.hash()[::-1]
sc.send(b'getdata', payload)

# look for merkleblock or tx commands and validate them
utxos = []
while True:
    envelope = sc.wait_for_commands({b'merkleblock\x00', b'tx\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'})
    if envelope.command.startswith(b'merkleblock'):
        stream = BytesIO(envelope.payload)
        mb = MerkleBlock.parse(stream)
        if not mb.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        stream = BytesIO(envelope.payload)
        t = Tx.parse(stream)
        for i, tx_out in enumerate(t.tx_outs):
            if tx_out.hash160() == h160:
                utxos.append((t.hash(), i))
    if len(utxos) > 2:
        break
for utxo in utxos:
    print(utxo[0].hex(), utxo[1])


version     : 7f1101000d04000000000000f18d785b00000000000000000000000000000000000000000000ffff4830fd33c7d10d04000000000000000000000000000000000000000000000000c5287ee3e8c03fa3102f5361746f7368693a302e31362e302fa61f150001
verack      : 
6411000000000000200000000000000020000000000000000000000000000010000000008010000020000000000000000000000000000200800000000100040000000000000000000008000000000000000010000080000008000000000010000000080030140000001400000000
sendheaders : 
sendcmpct   : 000200000000000000
sendcmpct   : 000100000000000000
ping        : 79d7056f970a6ed3
addr        : 01f18d785b0d0400000000000000000000000000000000ffffa2d49eac479d
feefilter   : e803000000000000
headers     : ...
merkleblock : 00000020d99f805a74a4684ac282598f528e180f5d3750c6221773cf71bf391400000000a3a3e821b80f8eca2e5ba1419483a2aa933344e9d0eff86f703e7004ca12d6bae51f765b3b275919f0ce6bb3c20d000001a3a3e821b80f8eca2e5ba1419483a2aa933344e9d0eff86f703e7004ca12d6ba0100
merkleblock : 0000002067c35e942bf50f02dd4b7cbb4baf952f

merkleblock : 000000204f6c479e9e1b7d2f6cd5034ef7296ebcc15f49299a59999a2700000000000000fd63277cf7d4c85df6c03eddc0cd80f831e99b5a09ff8ed6c775ed6f7183e31b8e24765b3b275919dcd8648ec30d000001fd63277cf7d4c85df6c03eddc0cd80f831e99b5a09ff8ed6c775ed6f7183e31b0100
merkleblock : 00000020d03e017328c745a807283a47734bfa74a2590949f1545c051d000000000000004bb283211d60d33d60543c9ed9ce0297a183fdde00656d3a4ed56d4a537421c5d724765b3b2759197c7e9b019e0d0000014bb283211d60d33d60543c9ed9ce0297a183fdde00656d3a4ed56d4a537421c50100
merkleblock : 00000020d3195090c072cb88e033baff3a171f9280b534470df5cdc2250000000000000049df1251fbb7b7b071ebcd13915808c2e06b6d70cd620c8d1c8d27517f209d7f3425765b3b275919a9995b29b70d00000149df1251fbb7b7b071ebcd13915808c2e06b6d70cd620c8d1c8d27517f209d7f0100
merkleblock : 00000020792535f3faddc3e1b1f9a7a81aa8bdd25185a24d3f977813340000000000000066e50a46df137a3cc5449fe9d3b369a3e0f21a1b5cf0f9f79f4fe8d80fa079c25725765b3b275919d8b63269bf0d00000166e50a46df137a3cc5449fe9d3b369a3e0f21a1b5cf0f9f79f4fe8d80